In [1]:
import requests
from bs4 import BeautifulSoup 
from pathlib import Path
import os
import sys
import time
import re
import traceback
import json

In [2]:
#https://www.gsaelibrary.gsa.gov/ElibMain/contractorList.do?contractorListFor=A

basicLink1="https://www.gsaelibrary.gsa.gov/ElibMain/"
basicLink2 = "contractorList.do?contractorListFor="   #seperate because basicLink1 is used by sub urls too.
bad_chars = "\\/:?*\"<>|"
bad_chars_list = ["\\","/",":","?","*","\"","<",">","|"]


pageLetter='A'

downloadPath = "D:\\mystuff\\Junaid Ahmed\\GSA scrap\\" 
dataDictionaryList_FilePath = "D:\\mystuff\\Junaid Ahmed\\GSA scrap\\DataDictionary" 
gsaList_FilePath = "D:\\mystuff\\Junaid Ahmed\\GSA scrap\\gsaList" 
noTableList_FilePath = "D:\\mystuff\\Junaid Ahmed\\GSA scrap\\noTableList" 



In [3]:
# Only run when you want to initialize the lists. Run if you are running for first time.
# These variables are the list and have complete data (dataDictionary variable and makeDictionary function for reference). remove these if memory is an issue.  
dataDictionaryList = []
gsaDictionaryList = []
noTableOrFileDictionaryList = []



In [4]:
def RemoveUnwwantedCharacters(myStr):
    myStr2 = ""
    for letter in myStr:
        if letter not in bad_chars_list:
            myStr2 += letter
    return myStr2

In [5]:
def GetLink(link):
    try:
        time.sleep(1)
        request=requests.get(link)
    except requests.exceptions.ConnectionError:
        print("Connection refused")
        print("Sleeping for 5 seconds")
        time.sleep(5)
        try:
            request=requests.get(link)
        except requests.exceptions.ConnectionError:
            print("Connection refused 2nd time.")
            print("Sleeping for 15 seconds")
            time.sleep(15)
            try:
                request=requests.get(link)
            except requests.exceptions.ConnectionError:
                print("Connection refused 3rd time.")
                print("Sleeping for 30 seconds")
                print("Link that is causing issue : " + str(link))
                time.sleep(30)
                try:
                    request=requests.get(link)
                except requests.exceptions.ConnectionError:
                    print("Connection refused 4th and last time. Will exit now")
                    print("Sleeping for 60 seconds")
                    time.sleep(60)
                    request=requests.get(link)
    return request

In [6]:
def downloadFile(url, fileName):
    with open(fileName, "wb") as file:
        response = requests.get(url)
        file.write(response.content)

In [7]:
# link = outer link, list wala page hai ye
# subLink = list k har item ka page ka link
# linkCounter 
# fileFinalURL
# linkText

def UpdateDictionaryList(dictList,dict_var):
    dictList.append(dict_var)                    # append doesnt return anything so it will just  append to refrenced list

def makeDictionary(link,linkText,linkCounter,subLink,isFileExist, fileFinalURL, contractorInformationHTML):
    dataDictionary = {}
    dataDictionary["Base Link"] = str(link)
    dataDictionary["Base Link Text"] = str(linkText)
    dataDictionary["Link Number In Base Link"] = str(linkCounter)
    dataDictionary["Specific Contract Information Link "] = str(subLink)
    dataDictionary["isFileExist"] = str(isFileExist)
    dataDictionary["File URL"] = str(fileFinalURL)
    dataDictionary["contractor Information Table HTML"] = str(contractorInformationHTML)
    return dataDictionary
    

In [8]:
def makeFolderIfNotExist(pageLetter,lastFolderName):
    Path(downloadPath+pageLetter+"\\"+folderName).mkdir(parents=True, exist_ok=True,mode=0o777)


In [9]:
def write_record(filePath,record):
    with open(filePath, 'w') as file:
        file.write(json.dumps(record,indent=2)) # use `json.loads` to do the reverse

def append_record(filePath,record):
    with open(filePath, 'a') as f:
        f.write(json.dumps(record, indent=2))
        
def read_record(filePath):
    with open(filePath) as f:
        my_list = [json.loads(line) for line in f]
    return my_list

In [10]:
def HandleSpecialLinkCases(caseName, caseText, link,linkText,linkCounter,subLink,isFileExist, fileFinalURL, contractorInformationHTML, pageLetter, folderName ):
    isFileExist = False
    print("\n-----------------------------------------------------------------------------------------")
    print(caseText)
    print("linkCounter : ",linkCounter)
    print("-----------------------------------------------------------------------------------------")

    
    dataDictionary = makeDictionary(link,linkText,linkCounter,subLink,isFileExist, fileFinalURL, contractorInformationHTML)
    
    if caseName == "GsaError":
        fileFinalURL ="http://www.gsa.gov/s2"
        UpdateDictionaryList(gsaDictionaryList,dataDictionary)  
    elif caseName = "NoRedirection"
    else:
        fileFinalURL = "No file/table avaiable"
        UpdateDictionaryList(noTableOrFileDictionaryList,dataDictionary)     
        
    makeFolderIfNotExist(pageLetter,folderName)
    destinationPath = (downloadPath+pageLetter+"\\"+folderName+"\\"+"content.json")
    write_record(destinationPath,dataDictionary)

In [11]:
# get the basic list page of  letter
link=basicLink1 + basicLink2 + pageLetter
r = GetLink(link)    
soup=BeautifulSoup(r.text,'html.parser')

In [ ]:

totalTime = 0
avgTime = 0


rows = soup.find("table", border=1).find_all("tr")    # get all rows

linkCounter = 0
processedLinkCounter = 0
try:
    print("Total Number of rows : " + str(len(rows)))
    for row in rows:             # traverse over all rows

        columns = row.find_all("td")
        for columnIndex,column in enumerate(columns):        

            linkCounter+=1
            print(linkCounter,end=", ")
            
            if linkCounter<=113  :     #set this value to the number of links already processed. set 0 if none. used in case error appeared and now want to resume instead of restart completely.
                continue
                
            processedLinkCounter+=1
            startTime = time.time()
            linkText = column.get_text()     # get the text of the link i.e name of the contractor
            folderName = RemoveUnwwantedCharacters (linkText)   # setting folder name as the contractor name minus not allowed characters (bad characters: definde above)
            subURL = row.find_all("td")[columnIndex].find('a',href=True)['href']

            subLink=basicLink1 + subURL        
            subRequest = GetLink(subLink)

            subSoup=BeautifulSoup(subRequest.text,'html.parser')

            try:
                contractorInformationHTML = subSoup.find_all("table")[10]
                FileTable = subSoup.find_all("table")[8].find_all("table")[7].find_all("tr")[1].find_all("td")[3].find_all("a")
            except:   #if there is no File table in contractor information
                caseText = "There is no table in sub link."
                caseName = "file_Table"                
                HandleSpecialLinkCases(caseName, caseText, link,linkText,linkCounter,subLink,isFileExist, fileFinalURL,
                                        contractorInformationHTML, pageLetter, folderName )
                continue
            
            if len(FileTable) == 0:    #if there is no File in contractor information
                caseText = "There is no file in the table"
                caseName = "file_Table"
                HandleSpecialLinkCases(caseName, caseText, link,linkText,linkCounter,subLink,isFileExist, fileFinalURL,
                                            contractorInformationHTML, pageLetter, folderName )
            
            else:                      # if there is  atleast one file
                
                for fileTablePart in FileTable:                    # there can be multiple files in one link so traverse over all of them
                    
                    FileHtmURL = fileTablePart["href"]
                    fileUrlShort = FileHtmURL.rsplit("/",1)[0]+"/"
                    
                    if (FileHtmURL == "http://www.gsa.gov/s2"):    # some file links are gsa.gov instead of htm links (htm links redirects to either pdf or docx file)
                        caseText = "File link is = http://www.gsa.gov/s2"
                        caseName = "GsaError"
                        HandleSpecialLinkCases(caseName,caseText, link,linkText,linkCounter,subLink,isFileExist, fileFinalURL, contractorInformationHTML, pageLetter, folderName )
                        
                    else:      # file exists and has htm url and that means it will have a file. need to go to htm url and meta tag for actual file url
                        
                        isFileExist = True            #does file exist? true if yes, No otherwise
                        fileFinalURL = FileHtmURL
                        dataDictionary = makeDictionary(link,linkText,linkCounter,subLink,isFileExist, fileFinalURL, contractorInformationHTML)
                        UpdateDictionaryList(dataDictionaryList,dataDictionary)                    
                        makeFolderIfNotExist(pageLetter,folderName)
                        destinationPath = (downloadPath+pageLetter+"\\"+folderName+"\\"+"content.json")

                        # For logging in filesystem
                        write_record(destinationPath,dataDictionary)
                        
                        # For downloading files
                        FileHtmRequest = GetLink(FileHtmURL)   #going to htm url , it redirects to file 
                        FileHtmSoup =BeautifulSoup(FileHtmRequest.text,'html.parser')      
                        FileHtmResult = FileHtmSoup.find("meta")["content"]                 # file url is in meta of htm file     
                        
                        if "url=" not in str(FileHtmResult):   # some links have file but their htm tags have text and do not redirect somewhere
                            caseText = "No redirects found. Its direct text."
                            caseName = "NoRedirection"
                            HandleSpecialLinkCases(caseName,caseText, link,linkText,linkCounter,subLink,isFileExist, fileFinalURL, contractorInformationHTML, pageLetter, folderName )

                        else:
                            FilemetaURL = FileHtmSoup.find("meta")["content"].split("url=")[1] #get the file url from middle url that redirects
                            redirectedFileURL = fileUrlShort+FilemetaURL            # arranging the file url.
                            extension = redirectedFileURL.split(".")[-1].lower()    # files are usually pdf or docx but code can handle any format
                            fileName = FileHtmURL.rsplit("/",2)[1] + "." +extension    # get the file name seperately and add extension to it. file name is at the end of  htm url
                            fileName = RemoveUnwwantedCharacters(fileName)      # path shouldnt contain bad characters. bad characters are define at the top

                            destinationPath = (downloadPath+pageLetter+"\\"+folderName+"\\"+fileName)
                            downloadFile(redirectedFileURL, destinationPath)
                        
            endTime = time.time()
            totalTime = totalTime + (endTime-startTime)
            avgTime = totalTime/processedLinkCounter

            if linkCounter%10==0:                               #General logging after processing 10 links
                print("\n\n Total links  = "+ str(linkCounter))
                print("Total links processed = "+ str(processedLinkCounter))
                print("Total elapsed time (in seconds) = "+str(totalTime))
                print("Average link time (in seconds)  = "+ str(avgTime))
                print("\n")
                
except Exception as e:
    print("\n\n General error occured. ");
    print("last instances of below items are : ")
    print("linkCounter = " + str(linkCounter))
    print("pageLetter = " + str(pageLetter))
    print("link = " + str(link))
    print("linkText = " + str(linkText))
    print("subLink = " + str(subLink))
    print("fileFinalURL = " + str(fileFinalURL))
    print("folderName = " + str(folderName))
#     print("fileName = " + str(fileName))
    print("destinationPath = " + str(destinationPath))
    print("Error : ")
    print(e)
    print("Traceback: ")
    print(traceback.format_exc())
    
    

Total Number of rows : 572
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 
-----------------------------------------------------------------------------------------
No redirects found. Its direct text.
linkCounter :  114
-----------------------------------------------------------------------------------------
115, 116, 117, 118, 119, 120, 

 Total links  = 120
Total links processed = 7
Total elapsed time (in seconds) = 87.00540351867676
Average link time (in seconds)  = 12.429343359810966


121, 122, 123, 124, 
--------------------------------------------------------------------

213, 214, 215, 
-----------------------------------------------------------------------------------------
There is no file in the table
linkCounter :  215
-----------------------------------------------------------------------------------------
216, 
-----------------------------------------------------------------------------------------
There is no file in the table
linkCounter :  216
-----------------------------------------------------------------------------------------
217, 218, 219, 220, 

 Total links  = 220
Total links processed = 107
Total elapsed time (in seconds) = 970.7279617786407
Average link time (in seconds)  = 9.072223941856455


221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 

 Total links  = 230
Total links processed = 117
Total elapsed time (in seconds) = 1084.379174232483
Average link time (in seconds)  = 9.268198070363102


231, 232, 233, 
-----------------------------------------------------------------------------------------
There is no file in the table
li

In [ ]:
#start from 114

# saving these variables in filesystem
write_record(dataDictionaryList_FilePath,dataDictionaryList)
write_record(gsaList_FilePath,gsaDictionaryList)
write_record(noTableList_FilePath,noTableOrFileDictionaryList)
